In [ ]:
# To test slideshow run this cell.  Use LiveReveal.js in class.
!jupyter nbconvert Lecture-19.ipynb --to slides --post serve

# Lecture 19:  Numerical Solutions to the Diffusion Equation (Spectral Method)

## What to Learn

* Fourier amplitudes and basis functions
* Periodic domains
* FFT
* Finite differencing
* Data structures and visualization

## What to do?

* Item 1
* Item 2

### Introduction
----

The spectral method

The author Boyd (Title: Chebyshev and Fourier Spectral Methods) provides a [graphical representation](./images/Boyd_Table.png) of the choices in basis functions and boundary conditions used in spectral methods in general.  In this lecture we focus on Fourier methods.  In this lecture we will focus on Fourier methods.

### Developing the Logic of the Spectral Method
----

The spectral method begins with the assumption that the solution to a PDE (the function $c(x,t)$) can be represented as a series expansion that contains time dependent amplitudes (the $a_k(t)$ factors) and spatially varying functions (the $\phi_k(x)$ terms are unit Fourier vectors).  This can be written as:

$$
c(x,t) = \sum_{k=0}^N a_k(t)\phi_k(x)
$$

Note the similarity to the approach taken in the separation of variables method.  Choosing the unit vectors from an orthonormal set (like the Fourier series) permits decoupling of the equations into $N$ independent equations that can be solved individually and simultaneously.  Furthermore, the assumption of the orthonormal set allows the time dependence to be placed within the amplitudes so that the independent variables are seperated.  Using this assumption for this lecture we will refer to the $a_k(t)$ as the amplitudes of the basis vectors, $\phi_m(x)$.  For this lecture, no restriction on the value of $N$ is imposed although practical considerations that relate to diffusive problems and the "smoothness" of the solutions will limit $N$ to a few terms.  The form of $\phi$ depends on the problem being solved; the boundary conditions and the initial conditions will affect this choice.  

Our example problem for this lecture is stated as follows:

$$
\frac{\partial c(x,t)}{\partial t} = \frac{\partial^2 c(x,t)}{\partial x^2}
$$

with the initial and boundary conditions:

$$
c(0,t) = 0\\
c(L,t) = 0\\
c(x,0) = c_0(x)
$$

We proceed by substituting our series expansion into the PDE.

The problem requires that our boundary value for the $c(x,t)$ are zero.  A choice involving a series of $\sin$ terms is therefore the appropriate basis functions to choose.  The argument of the $\sin$ function guarantees that the basis function (the Fourier vector) will be zero at $0$ and $L$ with the understanding that this choice enforces periodicity on the problem:

$$
\phi(x) = \sin \left( \frac{k\pi x}{L} \right)
$$ 

In [ ]:
import sympy as sp
x, y, z, t = sp.symbols('x y z t')
k, m, n = sp.symbols('k m n', integer=True)
f, g, h = sp.symbols('f g h', cls=sp.Function)
sp.var('a_k, phi, c', cls=sp.Function);
sp.var('L', real=True);
sp.init_printing();

In [ ]:
elementK = sp.Eq(c(x,t),a_k(t)*sp.sin(k*sp.pi*x/L))
elementK

Now let us differentiate this function as per the PDE above:

In [ ]:
sp.sin(sp.pi*k*x/L).diff(x,2)

In [ ]:
spaceDeriv = elementK.rhs.diff(x,2)
spaceDeriv

In [ ]:
timeDeriv = elementK.rhs.diff(t,1)
timeDeriv

Our final differential equation represented in $a(t)$ is therefore:

$$
\sum_{k=0}^N \sin{\left (\frac{\pi x}{L} k \right )} \frac{d a_k{\left (t \right )}}{d t}  = - \sum_{k=0}^N \frac{\pi^{2} k^{2}}{L^{2}} a_k{\left (t \right )} \sin{\left (\frac{\pi x}{L} k \right )}
$$

The approach from this point is to examine the behavior of the time dependent amplitudes in this resulting equation.  I rationalize this in the following way:

* I don't divide out or eliminate the basis vectors from consideration - algabraically that would be dividing by zero at some point within the domain.  Instead - I allow all basis vectors to exist at every time and point within the solution.
* I focus on the time evolution of the amplitudes - effectively permitting me to treat the vector of amplitudes as a linear set of ODEs.
* If an amplitude goes to zero - then that basis vector is no longer contributing to the solution.
* I recognize that the principle of superposition permits me to use one or infinitely many $k$-vectors in summation and that any individual $k$ or sum of $k$'s is also a solution to the problem.
* The initial amplitudes are determined by the initial condition $c(x,0)$.

One strategy is to develop a set of symbols so that we can solve for the amplitude at the new time without making algebraic mistakes!  We define the following symbols and let `SymPy` help us with the algebra.

In [ ]:
dt, ai, aip1 = sp.symbols('dt, a^{i}_k, a^{i+1}_k')

In [ ]:
differenceEquation = sp.Eq((ai-aip1)/dt,((sp.pi**2*k**2*ai)/L**2))
differenceEquation

In [ ]:
sp.solveset(differenceEquation,aip1)

At this point the implementation should be straightforward.

#### Implementation of the Spectral Method

Standard imports.  Worth noting here that `SciPy` provides $\sin$ and $\cos$ transforms if Fourier isn't what you want!

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Here we create a linear space that represents the x domain of our problem.  Wise to set `L` here too.

In [ ]:
numPoints = 20
L = 1.0
dt = 0.0001
steps = 1000
# we have access to np.pi for $\pi$
x = np.linspace(0,L,numPoints)

c_old = np.zeros((numPoints), dtype='complex128')
c_new = np.zeros((numPoints), dtype='complex128')
a_old = np.zeros((numPoints), dtype='complex128')
a_new = np.zeros((numPoints), dtype='complex128')

Now we create the parts that will hold the amplitudes.  We will use the FFT so there is no need to create basis vectors since the functions `fft` and `ifft` will provide that for us.

In [ ]:
k = np.fft.fftfreq(numPoints, d=L/(numPoints-1))
k2 = k**2

# create an initial condition (simple function like x**2)
np.copyto(c_new, np.sin(np.pi*x/L))
# transform it (dft or sin transform)
np.copyto(a_new,np.fft.fft(c_new))

Instabilities will occur if the amplitudes do not decay at each timestep.  The problem is that the condition depends on the wavenumber - so a suitable $dt$ must be chosen that satisfies the most restrictive condition for the largest wavenumber.

In [ ]:
(dt*np.pi**2*k2)/L**2 > 1

In [ ]:
for i in range(steps):
    # swap pointers
    a_new, a_old = a_old, a_new
    # find new amplitudes
    np.copyto(a_new, a_old*(1-(dt*np.pi**2*k2)/L**2))
    
# inverse transform it
np.copyto(c_new, np.fft.ifft(a_new))

In [ ]:
fig = plt.figure()

axes = fig.add_axes([0.1, 0.1, 0.8, 0.8]) # left, bottom, width, height (range 0 to 1)
axes.plot(x, c_new.real, 'r')

# Setting the y-limit cleans up the plot.
axes.set_ylim([0.0,1.0])
axes.set_xlabel('Distance $x$')
axes.set_ylabel('Concentration $c(x,t)$')
axes.set_title('Concentration Profile solved by Spectral Method');

[Top of Page](#Sections)

### Reading Assignments and Practice
----

Here are two short pieces of code.  Can you make sense of the implementation?  These came from https://open.umich.edu/find/open-educational-resources/literature-science-arts/parallel-spectral-numerical-methods.  The materials are provided under a Creative Commons license with attribution to the original authors whose names can be found at the above link.

In [ ]:
# %load Heat_Eq_1D_Spectral_BE.py
#!/usr/bin/env python
"""
Solving Heat Equation using pseudospectral methods with Backwards Euler:
u_t= \alpha*u_xx
BC = u(0)=0 and u(2*pi)=0 (Periodic)
IC=sin(x)
"""

import math
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator

# Grid
N = 64; h = 2*math.pi/N; x = [h*i for i in xrange(1,N+1)]

# Initial conditions
v = [math.sin(y) for y in x]
alpha = 0.5
t = 0     
dt = .001 #Timestep size

# (ik)^2 Vector
I = complex(0,1)
k = numpy.array([I*n for n in range(0,N/2) + [0] + range(-N/2+1,0)])
k2=k**2;

# Setting up Plot
tmax = 5.0; tplot = 0.1
plotgap= int(round(tplot/dt))
nplots = int(round(tmax/tplot))
data = numpy.zeros((nplots+1,N))
data[0,:] = v
tdata = [t]

for i in xrange(nplots):
    v_hat = numpy.fft.fft(v)  # convert to fourier space
    for n in xrange(plotgap):
        v_hat = v_hat / (1-dt*alpha*k2) # backward Euler timestepping

    v = numpy.fft.ifft(v_hat)   # convert back to real space
    data[i+1,:] = numpy.real(v)   # records data

    t = t+plotgap*dt    # records real time
    tdata.append(t)

# Plot using mesh
xx,tt = (numpy.mat(A) for A in (numpy.meshgrid(x,tdata)))

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(xx, tt, data,rstride=1, cstride=1, cmap=cm.jet,
        linewidth=0, antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.xlabel('x')
plt.ylabel('t')
plt.show()

In [ ]:
# %load Heat_Eq_1D_Spectral_FE.py
#!/usr/bin/env python
"""
Solving Heat Equation using pseudo-spectral and Forward Euler
u_t= \alpha*u_xx
BC= u(0)=0, u(2*pi)=0
IC=sin(x)
"""

import math
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator

# Grid
N = 64                     # Number of steps
h = 2*math.pi/N                 # step size
x = h*numpy.arange(0,N)    # discretize x-direction

alpha = 0.5                # Thermal Diffusivity constant
t = 0
dt = .001

# Initial conditions 
v = numpy.sin(x)
I = complex(0,1)
k = numpy.array([I*y for y in range(0,N/2) + [0] + range(-N/2+1,0)])
k2=k**2;

# Setting up Plot
tmax = 5; tplot = .1;
plotgap = int(round(tplot/dt))
nplots  = int(round(tmax/tplot))

data = numpy.zeros((nplots+1,N))
data[0,:] = v
tdata = [t]

for i in xrange(nplots):
    v_hat = numpy.fft.fft(v)

    for n in xrange(plotgap):
        v_hat = v_hat+dt*alpha*k2*v_hat # FE timestepping

    v = numpy.real(numpy.fft.ifft(v_hat))   # back to real space
    data[i+1,:] = v

    # real time vector
    t = t+plotgap*dt
    tdata.append(t)

# Plot using mesh
xx,tt = (numpy.mat(A) for A in (numpy.meshgrid(x,tdata)))

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(xx, tt, data,rstride=1, cstride=1, cmap=cm.jet,
        linewidth=0, antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.xlabel('x')
plt.ylabel('t')
plt.show()

[Top of Page](#Sections)